In [ ]:
import ccxt
import time
import pandas as pd
import plotly.express as px
import pyarrow.parquet as pq
import pyarrow as pa
from datetime import datetime, timedelta
import ccxt

# Создаем объекты бирж
#bitfinex = ccxt.bitfinex()
binance = ccxt.binance()
gateio = ccxt.gateio()
okx = ccxt.okx()
bybit = ccxt.bybit()

# Список бирж для проверки
#exchanges = [bitfinex, binance, gateio, okx, bybit]
exchanges = [binance, gateio, okx, bybit]

# Список таймфреймов для выгрузки данных
#timeframes = ['5m', '15m', '30m', '1h', '4h', '12h', '1d', '1w']
timeframes = ['5m']


# Функция для проверки наличия пары на бирже и получения данных
def get_data_from_exchange(exchange, pair, timeframe):
    try:
        markets = exchange.load_markets()  # Получаем список доступных рынков
        if pair in markets:
            print(f"Пара {pair} найдена на {exchange.id}. Загружаем данные...")
            return fetch_max_data(exchange, pair, timeframe)
        else:
            print(f"Пара {pair} не найдена на {exchange.id}.")
            return None
    except Exception as e:
        print(f"Ошибка при проверке рынка на {exchange.id}: {e}")
        return None

# Функция для получения данных за максимальный возможный период
def fetch_max_data(exchange, pair, timeframe):
    since = exchange.parse8601('2017-01-01T00:00:00Z')  # Начальная дата (можно изменять)
    now = exchange.milliseconds()  # Текущее время

    all_data = []
    
    while since < now:
        try:
            # Загружаем данные
            data = exchange.fetch_ohlcv(pair, timeframe, since)
            if not data:
                break
            all_data += data
            since = data[-1][0]  # Обновляем время для следующего запроса
            time.sleep(exchange.rateLimit / 1000)  # Задержка для соблюдения лимита запросов
        except ccxt.NetworkError as e:
            print(f"Ошибка сети для {pair} на {exchange.id}: {e}. Пауза и повтор.")
            time.sleep(5)  # Задержка перед повторной попыткой в случае сетевой ошибки
        except ccxt.ExchangeError as e:
            print(f"Ошибка биржи для {pair} на {exchange.id}: {e}")
            break
        except Exception as e:
            print(f"Неизвестная ошибка для {pair} на {exchange.id}: {e}")
            break

    return all_data

# Основная функция, которая последовательно проверяет все биржи
def get_max_data(pair, timeframe):
    for exchange in exchanges:
        data = get_data_from_exchange(exchange, pair, timeframe)
        if data:
            return data
        print(f"Пробуем следующую биржу для {pair}...")
    print(f"Данные для {pair} не найдены ни на одной из бирж.")
    return None

# Пример использования
pair = 'BTC/USDT'  # Укажите нужную пару
for timeframe in timeframes:
    print(f"Загружаем данные для {pair} с таймфреймом {timeframe}...")
    data = get_max_data(pair, timeframe)
    if data:
        print(f"Загружено {len(data)} записей для {pair} с таймфреймом {timeframe}")
    else:
        print(f"Данные для {pair} не найдены на указанных биржах.")


In [23]:
# Создаем объект биржи
exchange = ccxt.binance()

In [24]:
# Задаем список пар и таймфрейм
trading_pairs = [
    "BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT", "DOGE/USDT", "ADA/USDT", "TRX/USDT",
    "AVAX/USDT", "SHIB/USDT", "DOT/USDT", "LINK/USDT", "BCH/USDT", "NEAR/USDT", "MATIC/USDT", "LTC/USDT",
    "UNI/USDT", "PEPE/USDT", "/BTC/ETH/BTC", "BNB/BTC", "SOL/BTC", "XRP/BTC", "DOGE/BTC", "ADA/BTC", "TRX/BTC",
    "AVAX/BTC", "DOT/BTC", "LINK/BTC", "BCH/BTC", "NEAR/BTC", "MATIC/BTC", "LTC/BTC", "UNI/BTC", "SOL/BNB",
    "XRP/BNB", "ADA/BNB", "TRX/BNB", "AVAX/BNB", "DOT/BNB", "LINK/BNB", "BCH/BNB", "NEAR/BNB", "MATIC/BNB",
    "LTC/BNB", "XRP/ETH", "SOL/ETH", "ADA/ETH", "TRX/ETH", "AVAX/ETH"
]

In [25]:
# Таймфрейм 5 минут и за последние 2 года
timeframe = '5m'
since = exchange.parse8601((datetime.now() - timedelta(days=1460)).strftime('%Y-%m-%dT%H:%M:%SZ'))
now = exchange.milliseconds()  # Текущее время

### Функция для проверки целостности данных, проверки выбросов и пропусков

In [26]:
def check_data_integrity(df):
    """
    Проверяет целостность данных, наличие пропусков и выбросов.
    """
    print("Проверка пропусков...")
    missing_values = df.isnull().sum()
    print(missing_values)
    if missing_values.any():
        print("Пропуски найдены в следующих колонках:")
        print(missing_values[missing_values > 0])
    else:
        print("Пропусков не обнаружено.")

    print("\nПроверка на выбросы...")
    z_scores = np.abs((df[['open', 'high', 'low', 'close']] - df[['open', 'high', 'low', 'close']].mean()) / df[['open', 'high', 'low', 'close']].std())
    outliers = (z_scores > 3).sum()
    print(outliers)
    if outliers.any():
        print("Выбросы обнаружены!")
    else:
        print("Выбросов не обнаружено.")

### Объединение функции для проверки данных перед сохранением

In [27]:
def save_multiple_to_csv(data_dict, filename_prefix):
    """
    Сохраняет несколько наборов данных в CSV с проверкой данных перед сохранением.
    """
    for symbol, df in data_dict.items():
        check_data_integrity(df)  # Проверка данных перед сохранением
        df.to_csv(f'{filename_prefix}_{symbol}.csv', index=False)
        print(f"Данные для {symbol} сохранены в {filename_prefix}_{symbol}.csv")

In [28]:
def fetch_data(pair, timeframe, since, now):
    all_ohlcv = []
    while since < now:
        try:
            # Загружаем OHLCV данные (Open, High, Low, Close, Volume)
            ohlcv = exchange.fetch_ohlcv(pair, timeframe, since, limit=1000)
            if len(ohlcv) == 0:
                break
            all_ohlcv.extend(ohlcv)
            # Обновляем "since" для следующего запроса
            since = ohlcv[-1][0] + 1
            # Делаем паузу, чтобы не перегружать API биржи
            time.sleep(exchange.rateLimit / 1000)
        except Exception as e:
            print(f"Error fetching data for {pair}: {e}")
            break
    return all_ohlcv

def save_to_parquet(data, pair):
    # Преобразуем в DataFrame
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Сохраняем в Parquet
    table = pa.Table.from_pandas(df)
    pq.write_table(table, f"{pair.replace('/', '_')}.parquet")

    return df

# Функция для построения графика
def plot_data(df, pair):
    # Построение графика цен закрытия
    fig = px.line(df, x='timestamp', y='close', title=f'Close Price of {pair}')
    # Открытие графика в браузере
    fig.show(renderer="browser")

In [ ]:
for pair in trading_pairs:
    print(f"Fetching data for {pair}...")
    data = fetch_data(pair, timeframe, since, now)
    
    if data:
        print(f"Saving data for {pair}...")
        df = save_to_parquet(data, pair)
        
        print(f"Plotting data for {pair}...")
        plot_data(df, pair)
    else:
        print(f"No data fetched for {pair}")